In [4]:
import numpy as np
import pandas as pd
import math, os

In [5]:
df = pd.read_csv('Groceries.csv')

In [6]:
commodity_list = df['items'].values

In [7]:
for i in range(len(commodity_list)):
    commodity_list[i] = set(commodity_list[i][1 : -1].split(','))

In [8]:
# 创建初始的每一个物品单独作为项集的项集列表
def createInit(data):
    st = []
    for commodity_set in data:
        for item in commodity_set:
            if [item] not in st:
                st.append([item])
    st.sort()
    return list(map(frozenset, st))

In [26]:
# 扫描数据集，过滤支持度小于阈值的项集
def scanDataset(data, commodity_set, min_support):
    cnt = {}
    # 对项集在数据集行中出现次数进行计数
    for commodity_list in data:
        for line in commodity_set:
            if line.issubset(commodity_list):
                if not cnt.__contains__(line):
                    cnt[line] = 1
                else:
                    cnt[line] += 1
    sum = np.float(len(data))
    eligible_set = [] # 合格的项集
    support_data = {} # 项集到支持的映射
    for key, value in cnt.items():
        support =  value / sum
        if support >= min_support:
            eligible_set.append(key)
            support_data[key] = support
    return eligible_set, support_data

In [10]:
# 由旧的频繁集生成每个集合物品个数+1的新频繁集
def generateNewSet(old_set, Len):
    new_list = []
    length = len(old_set)
    for i in range(length):
        for j in range(i + 1, length):
            l1, l2 = sorted(list(old_set[i]))[ : Len - 2], sorted(list(old_set[j]))[ : Len - 2]
            if l1 == l2:
                new_list.append(old_set[i] | old_set[j])
    return new_list

In [11]:
def apriori(data, min_support):
    init_set = createInit(data)
    D = list(map(set, data))
    eligible_set, support_data = scanDataset(data, init_set, min_support)
    L = [eligible_set]
    k = 2
    while len(L[k - 2]) > 0:
        nxt = generateNewSet(L[k - 2], k)
        lk, supportData = scanDataset(data, nxt, min_support)
        support_data.update(supportData)
        L.append(lk)
        k += 1
    return L, support_data

In [12]:
# 验证
data = [{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]
L, support_data = apriori(data, 0.5)


In [107]:
def calConfidence(freq_set, h, support_data, rule_map, min_conf=0.5):
    rules = []
    for seq in h:
        if 'whole milk' in seq and 'root vegetables' in freq_set - seq and 'butter' in freq_set - seq:
            print(freq_set - seq, ' ', seq)
            print(support_data[freq_set], ' ', support_data[freq_set - seq]) 
        confidence = support_data[freq_set] / support_data[freq_set - seq]
        print(freq_set - seq, ' --> ', seq, ' ', confidence)
        if confidence >= min_conf:
            rule_map.append((freq_set - seq, seq, confidence))
            rules.append(seq)
    return rules

In [67]:
def rulesFromConsequnce(freq_set, h, support_data, rule_map, min_conf=0.5):
    length = len(h[0]) # h是可以放在规则右边的词汇集合
    if len(freq_set) > length + 1:
        nxt = generateNewSet(h, length + 1)
        nxt = calConfidence(freq_set, nxt, support_data, rule_map, min_conf)
        if len(nxt) > 1:
            rulesFromConsequnce(freq_set, nxt, support_data, rule_map, min_conf)

In [68]:
def generateRules(L, support_data, min_conf=0.5):
    total_rules = []
    for i in range(1, len(L)):
        for freq_set in L[i]:
            h = [frozenset([item]) for item in freq_set]
            if i > 1:
                rulesFromConsequnce(freq_set, h, support_data, total_rules, min_conf)
            else:
                calConfidence(freq_set, h, support_data, total_rules, min_conf)
    return total_rules

In [140]:
def generaterules(L, support_data, min_conf=0.5):
    total_rules = []
    for i in range(1, 3):
        for freq_set in L[i]:
            for item in freq_set:
                right = frozenset([item])
                confidence = support_data[freq_set] / support_data[freq_set - right]
                if confidence >= min_conf:
                    total_rules.append((freq_set - right, right, confidence))
    return total_rules

In [27]:
L, support_data = apriori(commodity_list, 0.005)

In [141]:
ans = generaterules(L, support_data)

In [39]:
ans_map = {}
for i in range(1, 4):
    ans_map[i] = []
for key, val in support_data.items():
    if len(key) <= 3:
        ans_map[len(key)].append((key, val))

In [147]:
with open('ans.txt', 'w') as w:
    for i in range(1, 4):
        w.write('-------------L' + str(i) + '-------------' + '\n')
        for tp in ans_map[i]:
            w.write('[')
            for word in tp[0]:
                w.write(word + ', ')
            w.write(']  ')
            w.write(str(tp[1]) + '\n')
            # w.write(str(tp[0]) + '  ' + str(tp[1]) + '\n'
        w.write('total: ' + str(len(ans_map[i])) + '\n')
    for grp in ans:
        w.write('[')
        for word in grp[0]:
            w.write(word + ',')
        w.write('] --> ')
        for word in grp[1]:
            w.write('[' + word + ']')
        w.write(' conf: ' + str(grp[2]) + '\n')
    w.write(str(len(ans)))